# Akori Mosaic Experiment Results

## Contents
* [Introduction](#intro)
* [Mosaic rate](#mosaic_rate)

<a id='intro'></a>
## Introduction

I make graphs, as correction material for the publication without explaining why I do this stuff.

<a id='mosaic_rate'></a>
## Mosaic rate

Calculate the mosaic change rate over time. Subjects could have a behavioural bias to move faster or slower over time.

In [ ]:
import scripts.data_loader as load
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scripts.qol as qol
%matplotlib inline
plt.rcParams['figure.dpi'] = 100  # image quality

# make a list of subjects: "pup01.json", "pup02.json", ..., "pup28.json"
bucket = "processed_mosaic_experimental_data"
dataset_dir = ["pup"+str(num+1).zfill(2)+".json" for num in range(28)] 

# for every image we calculate in a time window how many times subject travels to a mosaic
mosaic_time = 12000  # 12 seconds
window = 2000
images = 84
steps = range(0, mosaic_time, window)

object_rate = np.zeros((len(dataset_dir), len(steps)), dtype=float)
for i_subject, subject in enumerate(dataset_dir):
    subject_fixations = load.fixtimes(subject, bucket)  # loads |t_on|t_off|image|object|
    
    subject_object_rate = np.zeros(len(steps))
    # TODO: for now we use t0 == t_on(0) but it's not exact 
    for image in range(images):  # we make (t_on-t0, object) array f/e image
        
        subfix = []
        for a_fix in subject_fixations:
            if a_fix[2] == image+1:
                subfix.append([a_fix[0], a_fix[3]])
        subfix = np.array(subfix)
        # make first fixation start at 0
        t0 = subfix[0, 0]
        for i in range(len(subfix)):
            subfix[i, 0] = subfix[i, 0] - t0
        
        # now calculate "transition rate" 
        image_object_rate = np.zeros(len(steps), dtype=float)
        current_object = subfix[0, 1]
        
        # we count an object change as a transition and add it to the step window
        for a_fix in subfix:
            if a_fix[1] != current_object:
                step_index = int(a_fix[0] / window)
                image_object_rate[step_index] = image_object_rate[step_index] + 1.0
                current_object = a_fix[1]
        
        # finally add the image rate to the total for the subject
        subject_object_rate += image_object_rate
    
    # we added all the image rates so now we divide by total and add it to the grand matrix
    subject_object_rate /= images
    object_rate[i_subject, :] = subject_object_rate
    
# Now that we have transition rates for every subject we can calculate average signal
grand_average = object_rate.mean(axis=0)
grand_std = object_rate.std(axis=0)

# Let's calculate ANOVA to check if means from each step are different or not
# TODO: It's implicitly hardcoded for now we have 6-steps but this can be made automatically...later
anova = stats.f_oneway(object_rate[:, 0], object_rate[:, 1], object_rate[:, 2], object_rate[:, 3], 
                       object_rate[:, 4], object_rate[:, 5])
print(anova) 

# Let's plot!
fig, ax = plt.subplots()
rect = ax.bar(steps, grand_average, yerr=grand_std, width=1000)
ax.set_xlabel('Mosaic time window [milliseconds]')
ax.set_ylabel('Rate [count]')
ax.set_title('Transition count for a time-window during image exploration')

plt.tight_layout()
plt.show()